In [53]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [54]:
from data import * 

In [55]:
solver = pywraplp.Solver.CreateSolver('GLOP')
if not solver:
    exit

In [56]:
foods = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in data]

In [57]:
# Create the variables x and y.
x = solver.NumVar(0, solver.infinity(), 'x')
y = solver.NumVar(0, solver.infinity(), 'y')
# The parameters of NumVar are:

In [58]:
print('Number of variables =', solver.NumVariables())

Number of variables = 79


In [59]:
# Let's have a look at one example 0 <= 2*x + y <= 5.
ct = solver.Constraint(0, 5, 'ct')
ct.SetCoefficient(x, 2)
ct.SetCoefficient(y, 1)

In [60]:
# The constrainsts can be reprsented as arrays to make evrything easier
# Create the constraints, one per nutrient.
constraints = []
for i, nutrient in enumerate(nutrients):
    # Append one constrint per nutrient. If you look at the nutrient table,
    # you will see that the minimum value is stored at nurient[1] (second column)
    constraints.append(solver.Constraint(nutrient[1], solver.infinity()))

    # The coefficients in this constraint are given by the 'data' table
    # The variable for which we are creating the coefficient is foods[j]
    # The coefficient itself has the value of the corresponding nutrient in 'data'
    # table, which is located at position i + 3 (where i is the nutrient number)
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i + 3])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 10


In [61]:
# Finally, we set the objective function
# As usual, let's see in comments a simple objective function for two variables
# Again, we specify the coefficients of each variable for the linear combination of all variables
# Create the objective function, 3 * x + y.
objective = solver.Objective()
objective.SetCoefficient(x, 3)
objective.SetCoefficient(y, 1)
objective.SetMaximization()

In [62]:
# Let's translate this to our problem
# Objective function: Minimize the sum of foods cost. 
# The objective function is the sum of each variable (food quantity) multiplied 
# by its cost (cost is in column 3). The costs are the coefficients
objective = solver.Objective()
for f, food in enumerate(foods):
    objective.SetCoefficient(food, data[f][2]) # 2 is column 3
objective.SetMinimization()

status = solver.Solve()

In [63]:
# Check that the problem has an optimal solution.
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)
else:
    print('Optimal solution found')

Optimal solution found


In [64]:
# Display the amounts to purchase of each food.
nutrients_result = [0] * len(nutrients)
print('\nDaily Foods:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        print('{}: {}'.format(data[i][0], food.solution_value()))
        for j, _ in enumerate(nutrients):
            nutrients_result[j] += data[i][j + 3] * food.solution_value()

print('\nOptimal cost: ${:.4f}'.format(objective.Value()))



Daily Foods:
Lard: 47.75255356126264
Spinach: 5.340065500821445
Sweet Potatoes: 0.2962962962962963

Optimal cost: $512.7407


In [67]:
print('\nNutrients per day:')
for i, nutrient in enumerate(nutrients):
    print('{}: (Max {:.3f}) (min {})'.format(nutrient[0], nutrients_result[i], nutrient[1]))

#Calories and nutrients do not line up - the min calories and min nutrients do but the max calories do not. 
#You cannot have a max of 2000 and a min of 2000 calories



Nutrients per day:
Calories (kcal): (Max 2000.000) (min 2000)
Protein (g): (Max 606.936) (min 70)
Calcium (g): (Max 0.800) (min 0.8)
Iron (mg): (Max 752.929) (min 12)
Vitamin A (KIU): (Max 5000.000) (min 5000)
Vitamin B1 (mg): (Max 32.927) (min 1.8)
Vitamin B2 (mg): (Max 99.169) (min 2.7)
Niacin (mg): (Max 439.578) (min 18)
Vitamin C (mg): (Max 15278.399) (min 75)
